In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torch transformers accelerate
!pip install pandas openpyxl
!pip install transformers peft torch


In [ ]:
pip install gradio


In [ ]:
import os
os.environ["HF_AUTH_TOKEN"] = "Your_Access_Token"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import os

base_model_id = "NousResearch/Llama-2-7b-hf"  # Public and compatible
adapter_model_id = 'Ashishkr/llama-2-medical-consultation'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load base model (standard float16 or 8-bit if needed)
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map='auto',
    torch_dtype=torch.float16,
    use_auth_token=os.environ["HF_AUTH_TOKEN"]
)

# Load LoRA adapter
model = PeftModel.from_pretrained(model, adapter_model_id).to(device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    use_auth_token=os.environ["HF_AUTH_TOKEN"]
)


In [ ]:
import pandas as pd

# Upload your file
df = pd.read_excel("/content/drive/MyDrive/Ayush/drug_interaction_dataset.xlsx")

# Create prompt format
def create_prompt(row):
    return (
        f"You are a medical expert. Analyze the following drug interaction and explain the likely outcome.\n"
        f"Drug A: {row['Drug_A']}\n"
        f"Drug B: {row['Drug_B']}\n"
        f"Mechanism: {row['Mechanism_of_Interaction']}\n"
        f"Outcome:"
    )

# Apply to dataset
df['prompt'] = df.apply(create_prompt, axis=1)

# Show one example prompt
print(df['prompt'].iloc[0])


In [ ]:
def generate_response(prompt, max_new_tokens=100, temperature=0.5):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from the output to keep just the model's answer
    cleaned = decoded.replace(prompt, "").strip()
    return cleaned


In [ ]:
import gradio as gr

def medllama_interaction_response(drug_a, drug_b, mechanism):
    # Use specific prompt structure
    prompt = (
    f"You are a medical expert. Analyze the following drug interaction and explain the likely outcome.\n"
    f"Drug A: {drug_a}\n"
    f"Drug B: {drug_b}\n"
    f"Mechanism: {mechanism if mechanism else 'Unknown'}\n"
    f"Outcome:"
)


    # Generate response using model
    response = generate_response(prompt)
    return response

# Define the Gradio interface
demo = gr.Interface(
    fn=medllama_interaction_response,
    inputs=[
        gr.Textbox(label="Drug A"),
        gr.Textbox(label="Drug B"),
        gr.Textbox(label="Mechanism of Interaction (e.g., Inhibits clearance)")
    ],
    outputs=gr.Textbox(label="Predicted Interaction Outcome"),
    title="🧠 MedLLaMA Drug Interaction Predictor",
    description="Enter two drugs and their mechanism of interaction. MedLLaMA will predict the likely outcome."
)

# Launch the Gradio app
demo.launch(share=True)
